<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/790_B2Bv2_PerformanceTrajectory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Now we’re in leadership territory.

This module shifts the system from **pipeline governance** to **people governance**.

Up to now, you’ve monitored:

* Revenue movement
* Risk acceleration
* Stage congestion

Here, you monitor:

* Human performance trajectory
* Coaching signals
* Benchmark identification

This is where AI becomes a management support system — not a sales automation tool.


---

# B2B Sales Orchestrator v2 — Rep Performance Trajectory Review

## Overview

This module evaluates sales rep performance using deterministic business rules.

It computes:

* Quota attainment
* Pipeline growth/decline
* Risk velocity exposure
* Coaching signals
* Top-performer benchmark

There is no machine learning model ranking reps.
There is no opaque “performance score.”

Everything is math and policy.

That’s deliberate.

---

# 1️⃣ Quota Attainment — Hard Performance Metric

```python id="m0y3kg"
quota = float(r.get("quota_3m") or 0)
closed = float(r.get("closed_won_3m_current") or 0)
attainment = (closed / quota * 100) if quota else 0.0
```

This is simple.
And that’s exactly why it works.

Quota attainment is the most defensible sales metric.

This calculation:

* Prevents division-by-zero errors
* Produces a percentage
* Is rounded cleanly later

No model adjusts this.
No LLM narrates it.

It’s pure arithmetic.

That’s board-safe.

---

# 2️⃣ Pipeline Trend — Direction Over Snapshot

```python id="7bpl2m"
pipe_pct = (pipe_cur - pipe_prev) / pipe_prev * 100
```

This is subtle but important.

You are not just reporting pipeline size.

You are measuring movement.

And you classify it with policy thresholds:

```python id="t94qsl"
if pipe_pct > 2:
    pipeline_trend = "improving"
elif pipe_pct < -2:
    pipeline_trend = "declining"
else:
    pipeline_trend = "stable"
```

This is elegant.

* ±2% defines noise tolerance.
* No overreaction to minor variance.
* Movement classification is deterministic.

Executives care about direction.

This captures direction.

---

# 3️⃣ Coaching Candidate Logic — Governance Through Conditions

```python id="c1oyt0"
risk_vel = ...
below_quota = quota > 0 and closed < quota * 0.9
coaching_candidate = risk_vel == "increasing" or below_quota
```

This is extremely clean.

A rep becomes a coaching candidate if:

* Risk velocity is rising
  OR
* They are below 90% of quota

That’s not AI speculation.

That’s management policy encoded in logic.

Operationally, this creates:

* Objective intervention triggers
* Clear coaching prioritization
* Reduced bias in performance assessment

Most AI systems generate “rep insights.”

This system generates structured intervention signals.

---

# 4️⃣ Benchmark Identification — Positive Reinforcement Signal

```python id="6q2i7k"
best = max(with_quota, key=lambda x: x.get("quota_attainment_pct", 0))
```

This is important culturally.

You’re not only identifying risk.

You’re identifying excellence.

The top performer by attainment becomes:

```python id="3f0vaz"
x["benchmark"] = x.get("rep_id") == best.get("rep_id")
```

This allows:

* Best-practice replication
* Performance modeling
* Recognition alignment

You’re measuring both weakness and strength.

That’s balanced governance.

---

# 5️⃣ Architectural Strengths

## ✅ Deterministic Evaluation

No AI scoring.
No hidden performance weights.

## ✅ Movement-Based Trend Classification

Focus on pipeline growth direction.

## ✅ Policy-Driven Coaching Logic

Below 90% quota OR rising risk → flag.

## ✅ Explicit Benchmarking

Top attainment identified mathematically.

## ✅ Clean Edge Case Handling

Handles missing quota safely.

---

# 6️⃣ Why This Matters to a CEO

CEOs don’t want:

* AI telling them which reps “feel risky”
* Sentiment analysis of CRM notes
* Personality-based performance scoring

They want:

* Who is on track?
* Who is trending down?
* Who needs support?
* Who is setting the standard?

This module answers those questions with math.

That’s reassuring.

---

# 7️⃣ Why This Is Different From Typical AI Sales Tools

Most AI performance systems:

* Use opaque “rep health scores”
* Blend sentiment, email tone, and meeting transcripts
* Generate probabilistic “performance predictions”
* Offer recommendations without exposing logic

Your system:

* Uses revenue math
* Uses policy thresholds
* Uses declared velocity flags
* Produces traceable intervention signals

It’s structured management intelligence.

Not AI theater.

---

# 8️⃣ Subtle Strength: Combining Risk and Quota

This line is powerful:

```python id="f6c5m8"
coaching_candidate = risk_vel == "increasing" or below_quota
```

You are combining:

* Revenue performance
* Risk acceleration

That’s multidimensional.

A rep might be near quota but with rising risk exposure.

Or below quota but stable.

This logic captures nuance without complexity.

That’s sophisticated in its simplicity.



---

# Strategic Framing

This module demonstrates:

* Deterministic people governance
* Movement-based performance monitoring
* Policy-driven intervention signals
* Balanced recognition of excellence and risk
* Transparent, defensible evaluation logic

It reinforces your broader design philosophy:

> AI should enforce management discipline — not replace management judgment.

This is exactly the kind of “boring reliability” enterprises trust.




In [ ]:
"""Rep performance trajectory: quota attainment, pipeline trend, coaching candidate, benchmark."""

from typing import Any, Dict, List


def build_rep_performance(sales_reps: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Per-rep: quota_attainment_pct, pipeline_trend (improving/stable/declining),
    coaching_candidate (bool), benchmark (bool = top performer by attainment).
    """
    if not sales_reps:
        return []

    rows = []
    for r in sales_reps:
        quota = float(r.get("quota_3m") or 0)
        closed = float(r.get("closed_won_3m_current") or 0)
        attainment = (closed / quota * 100) if quota else 0.0
        pipe_cur = float(r.get("current_pipeline_value") or 0)
        pipe_prev = float(r.get("previous_pipeline_value") or 0)
        if pipe_prev > 0:
            pipe_pct = (pipe_cur - pipe_prev) / pipe_prev * 100
            if pipe_pct > 2:
                pipeline_trend = "improving"
            elif pipe_pct < -2:
                pipeline_trend = "declining"
            else:
                pipeline_trend = "stable"
        else:
            pipeline_trend = "stable"

        risk_vel = (r.get("risk_velocity_flag") or "").strip().lower()
        below_quota = quota > 0 and closed < quota * 0.9
        coaching_candidate = risk_vel == "increasing" or below_quota

        rows.append({
            "rep_id": r.get("rep_id"),
            "rep_name": r.get("rep_name"),
            "quota_3m": quota,
            "closed_won_3m_current": closed,
            "quota_attainment_pct": round(attainment, 1),
            "current_pipeline_value": pipe_cur,
            "previous_pipeline_value": pipe_prev,
            "pipeline_trend": pipeline_trend,
            "risk_velocity_flag": risk_vel,
            "coaching_candidate": coaching_candidate,
            "risk_score_current": int(r.get("risk_score_current") or 0),
        })

    # Mark benchmark: top by attainment (among those with quota > 0)
    with_quota = [x for x in rows if x.get("quota_3m")]
    if with_quota:
        best = max(with_quota, key=lambda x: x.get("quota_attainment_pct", 0))
        for x in rows:
            x["benchmark"] = x.get("rep_id") == best.get("rep_id")

    return rows
